In [ ]:
# app0 and nv dumped
# 3ffbdbf3 in app0 is where real flag located in actual binary.
# TISC 2024 :: I2C IMPLANT

# Available commands:
# - SEND <bytes>     - Sends hex encoded bytes to I2C bus
# - RECV <num_bytes> - Receive num_bytes from I2C bus
# - EXIT             - Exit the program

# Example:
# > SEND 12 34       - Sends 0x12, 0x34 to I2C bus, which will be received by the device at addr 0x09 as a WRITE request and payload 0x34
# > RECV 4           - Attempts to receive 4 bytes from the I2C bus, if no slave device sends data, it will return 0s

# Read More: https://en.wikipedia.org/wiki/I%C2%B2C#Reference_design
# exact version kinda https://github.com/espressif/esp-idf/tree/1067b28707e527f177752741e3aa08b5dc64a4d7/

In [3]:
from Crypto.Util.number import bytes_to_long
from tqdm import tqdm
from pwnlib.tubes.remote import remote
trace = ""
r = remote("chals.tisc24.ctf.sg", 61622)
r.recvuntil(b"Read More")
r.recvuntil(b"> ")
def send_bytes(read: bool, addr: int, val: int):
    p = f"SEND {hex((addr<<1) + read)[2:].upper()} {hex(val)[2:].upper()}\n"
    trace+=p
    r.send(p.encode())
    trace+=r.recvuntil(b"> ").decode()
def recv_bytes(length: int):
    assert length <= 32, "recv command restricts byte length to 32"
    p = f"RECV {length}\n"
    trace+=p
    r.send(p.encode())
    out = r.recvuntil(b"> ").decode()
    trace+=out
    return bytes.fromhex(out.split("\n")[0])
# for i in tqdm(range(256)):
#     send_bytes(i, 0)
#     out = recv_bytes(32)
#     if (bytes_to_long(out)>0):
#         print(i, recv_bytes(32))
# send_bytes(0x4e << 1, 0x34)
# send_bytes(0x4d << 1, 0x34)
# send_bytes(0x7f << 1, 0x34)
# for j in tqdm(range(256)):
#     for i in (range(128)):
#         send_bytes(False, i, j)
#         out = recv_bytes(32)
#         if (bytes_to_long(out)>0):
#             print(i, out)

100%|██████████| 256/256 [09:42<00:00,  2.27s/it]


In [21]:
send_bytes(True, 0x13, 0X0F)

'0x9'

In [4]:
# 9A984347 at pos 0x1b in  otadata, rest is ff
# above cleared up is not important
# spiffs fully ff
# nvs unimportant info
# coredump fully ff
# app0 decompiled
# app1 error: "esptool.FatalError: Invalid firmware image magic=0x0"

# everything that matters is probably in app0/app1
# app0 uses https://github.com/espressif/esp-idf/tree/master

1073470451

In [13]:
hex(0x4e << 1)

'0x9c'

In [15]:
hex(0x7F << 1)

'0xfe'

In [1]:
hex(0x10000+1310720)

'0x150000'

In [2]:
app0 = open("./bin/app0_out.bin", 'rb').read()
app1 = open("./bin/app1_out.bin", 'rb').read()
open("./bin/app_out.bin", 'wb').write(app0+app1)

2621440

In [ ]:
# reading partition table...
# entry 0:
#   label      : nvs
#   offset     : 0x9000
#   length     : 20480
#   type       : 1 [DATA]
#   sub type   : 2 [WIFI]

# entry 1:
#   label      : otadata
#   offset     : 0xe000
#   length     : 8192
#   type       : 1 [DATA]
#   sub type   : 0 [OTA]

# entry 2:
#   label      : app0
#   offset     : 0x10000
#   length     : 1310720
#   type       : 0 [APP]
#   sub type   : 16 [ota_0]

# entry 3:
#   label      : app1
#   offset     : 0x150000
#   length     : 1310720
#   type       : 0 [APP]
#   sub type   : 17 [ota_1]

# entry 4:
#   label      : spiffs
#   offset     : 0x290000
#   length     : 1441792
#   type       : 1 [DATA]
#   sub type   : 130 [unknown]

# entry 5:
#   label      : coredump
#   offset     : 0x3f0000
#   length     : 65536
#   type       : 1 [DATA]
#   sub type   : 3 [unknown]

# MD5sum:
# 972dae2ff872a0142d60bad124c0666b
# Done

In [6]:
s_all = open('dat', 'r').readlines()


In [74]:
from glob import glob
# espidf_files = [f.split("\\")[-1] for f in glob("**/*.c", root_dir="bin/esp-idf/components", recursive=True)]
# example_files = [f.split("\\")[-1] for f in glob("**/*.c", root_dir="bin/esp-idf/examples", recursive=True)]
espidf_files_corr_version = [f.split("\\")[-1] for f in glob("**/*.c", root_dir="esp-idf/components", recursive=True)]
example_files_corr_version = [f.split("\\")[-1] for f in glob("**/*.c", root_dir="esp-idf/examples", recursive=True)]
dump_files = [s[1:-2].split("/")[-1] for s in s_all if s[1:-2].endswith(".c")]
out = [d for d in dump_files if 
       (d not in espidf_files_corr_version)
       and (d not in example_files_corr_version)
    ]
len(out)

37

In [75]:
out

['bootloader_flash_config.c',
 'esp_timer_esp32.c',
 'syscall_table.c',
 'locale.c',
 'rtc_periph.c',
 'assert.c',
 'vfprintf.c',
 'vfprintf.c',
 'vfprintf.c',
 'dtoa.c',
 'mprec.c',
 'vfprintf.c',
 'libgcc2.c',
 'libgcc2.c',
 'libgcc2.c',
 'libgcc2.c',
 'errno.c',
 'fiprintf.c',
 'fopen.c',
 'fseek.c',
 'fseeko.c',
 'mbtowc_r.c',
 'printf.c',
 'puts.c',
 'reent.c',
 'snprintf.c',
 'vprintf.c',
 'flags.c',
 'localeconv.c',
 's_frexp.c',
 'pm_esp32.c',
 'project_elf_src.c',
 'esp32-hal-uart.c',
 'esp_ipc.c',
 'esp_ipc_isr.c',
 'dtoa.c',
 'mprec.c']

['app_trace\\app_trace.c',
 'app_trace\\app_trace_util.c',
 'app_trace\\heap_trace_tohost.c',
 'app_trace\\host_file_io.c',
 'app_trace\\gcov\\gcov_rtio.c',
 'app_trace\\sys_view\\esp32\\SEGGER_RTT_esp32.c',
 'app_trace\\sys_view\\ext\\heap_trace_module.c',
 'app_trace\\sys_view\\ext\\logging.c',
 'app_trace\\sys_view\\Sample\\Config\\SEGGER_SYSVIEW_Config_FreeRTOS.c',
 'app_trace\\sys_view\\Sample\\OS\\SEGGER_SYSVIEW_FreeRTOS.c',
 'app_trace\\sys_view\\SEGGER\\SEGGER_SYSVIEW.c',
 'app_trace\\test\\test_trace.c',
 'app_update\\esp_app_desc.c',
 'app_update\\esp_ota_ops.c',
 'app_update\\test\\test_app_desc.c',
 'app_update\\test\\test_ota_ops.c',
 'app_update\\test\\test_switch_ota.c',
 'asio\\port\\src\\esp_asio_openssl_stubs.c',
 'bootloader\\subproject\\components\\micro-ecc\\uECC_verify_antifault.c',
 'bootloader\\subproject\\main\\bootloader_start.c',
 'bootloader_support\\src\\bootloader_clock_init.c',
 'bootloader_support\\src\\bootloader_clock_loader.c',
 'bootloader_support\\s

In [55]:
[s[1:-2] for s in s_all if s[1:-2].endswith(".c")]

['esp_app_desc.c',
 'cpu_start.c',
 'crosscore_int.c',
 'dport_access.c',
 'int_wdt.c',
 'intr_alloc.c',
 'system_api.c',
 'cache_err_int.c',
 'clk.c',
 'debug_helpers.c',
 'esp_timer.c',
 'freertos_hooks.c',
 'ipc.c',
 'cpu_util.c',
 'log.c',
 'heap_caps.c',
 'port.c',
 'locks.c',
 'reent_init.c',
 'time.c',
 'timer.c',
 'bootloader_flash_config.c',
 'cache_utils.c',
 'flash_mmap.c',
 'flash_ops.c',
 'esp_flash_api.c',
 'spi_flash_os_func_app.c',
 'esp_timer_esp32.c',
 'ringbuf.c',
 'multi_heap.c',
 'panic.c',
 'memspi_host_driver.c',
 'spi_flash_chip_generic.c',
 'spi_flash_chip_issi.c',
 'rtc_clk.c',
 'rtc_time.c',
 'spi_flash_hal_iram.c',
 'rtc_init.c',
 'rtc_wdt.c',
 'heap.c',
 'queue.c',
 'tasks.c',
 'timers.c',
 'xtensa_init.c',
 'xtensa_intr.c',
 'dbg_stubs.c',
 'eri.c',
 'list.c',
 'pthread_local_storage.c',
 'task_wdt.c',
 'FreeRTOS-openocd.c',
 'vfs.c',
 'vfs_uart.c',
 'syscall_table.c',
 'periph_ctrl.c',
 'rtc_module.c',
 'uart.c',
 'esp_efuse_table.c',
 'esp_flash_spi_init

In [41]:
glob("**/*.c", root_dir="bin/esp-idf/components", recursive=True)

['app_trace\\app_trace.c',
 'app_trace\\app_trace_membufs_proto.c',
 'app_trace\\app_trace_util.c',
 'app_trace\\debug_stubs.c',
 'app_trace\\heap_trace_tohost.c',
 'app_trace\\host_file_io.c',
 'app_trace\\gcov\\gcov_rtio.c',
 'app_trace\\port\\port_uart.c',
 'app_trace\\port\\riscv\\port.c',
 'app_trace\\port\\xtensa\\port.c',
 'app_trace\\sys_view\\esp\\SEGGER_RTT_esp.c',
 'app_trace\\sys_view\\ext\\heap_trace_module.c',
 'app_trace\\sys_view\\ext\\logging.c',
 'app_trace\\sys_view\\Sample\\FreeRTOSV10.4\\SEGGER_SYSVIEW_FreeRTOS.c',
 'app_trace\\sys_view\\Sample\\FreeRTOSV10.4\\Config\\esp\\SEGGER_SYSVIEW_Config_FreeRTOS.c',
 'app_trace\\sys_view\\SEGGER\\SEGGER_SYSVIEW.c',
 'app_trace\\test_apps\\main\\test_app_trace_main.c',
 'app_trace\\test_apps\\main\\test_trace.c',
 'app_update\\esp_ota_app_desc.c',
 'app_update\\esp_ota_ops.c',
 'app_update\\test_apps\\main\\app_main.c',
 'app_update\\test_apps\\main\\test_ota_ops.c',
 'app_update\\test_apps\\main\\test_switch_ota.c',
 'bootl